In [180]:
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
from sklearn.model_selection import train_test_split
import copy
import csv

In [181]:
TRANSLATION_PROMPT = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the OpenDaylight (ODL) SDN controller's configuration datastore.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

a. **Traffic Forwarding and Queue-Based Traffic Control Rule:** Define rules for forwarding traffic based on IPv4 destination, TCP/UDP ports, and optionally assign traffic to specific queues for prioritization. 
b. **Firewall and Blocking:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP).  
c. **Port-Based Forwarding:** Redirect traffic entering a specific port to another designated port.

### JSON STRUCTURE:

1. **Traffic Forwarding and Queue-Based Traffic Control Rule:**  

```json
{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": 2048
          }
        },
        "ipv4-source": "<ip_address/mask>", //**optional**
        "ipv4-destination": "<ip_address/mask>", //**optional**
        "ip-match": {
          "ip-protocol": <integer>
        },
        "tcp-destination-port": <integer>,
        "udp-destination-port": <integer>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "set-queue-action": {
                    "queue-id": <queue_id>
                  }
                },
                {
                  "order": 1,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

2. **Blocking or Dropping Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": <integer>
          }
        },
        "ipv4-source": "<ip_address/mask>",
        "ipv4-destination": "<ip_address/mask>"
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "drop-action": {}
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

3. **Port-Based Forwarding Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name_summerizing_the_intent>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "in-port": <port_number>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

Field Descriptions: 
id: A number representing a unique identifier for the flow (0 for default).
priority: Priority level (higher numbers indicate higher priority). For dropping or blocking or firewall rule, assign priority greater than 300.
table_id: An integer representing the flow table identifier (0 for default).
flow-name: A short, descriptive flow name that summerizes the intent.
hard-timeout: Timeout in seconds after which the flow is removed (0 for default).
idle-timeout: Timeout in seconds after which the flow is removed if there's no activity (0 for default).
ethernet-type: Ethernet protocol type (e.g., 2048 for IPv4).
ipv4-destination: IPv4 address in CIDR notation (e.g., 10.0.0.1/32). Note: This field is optional. Don't include it unless IP address (e.g., 10.0.0.1) is explicitly mentioned in the intent.
ipv4-source: Source IP address (optional). Note: This field is optional. Don't include it unless IP address (e.g., 10.0.0.1) is explicitly mentioned in the intent.
ip-protocol: Use "ip-match" and "ip-protocol" when specifying specific transport layer protocols (e.g., 6 for TCP, 17 for UDP, 1 for ICMP (optional)).
tcp-source-port: The source port for the connection, usually a random high port on the client (rarely fixed) (optional).
tcp-destination-port: The destination port for the connection (i.e., the server port, e.g., 80 for HTTP) (optional).
udp-source-port: UDP port number (optional).
udp-destination-port: UDP port number (optional).
in-port: A value representing incoming interface port number (optional).
output-action: Use "output-action" and "output-node-connector" to specify the output port number (optional).
set-queue-action: Use "set-queue-action" and "queue-id" when the intent specifies assigning traffic to a queue (The "queue-id" is an integer (0 for default)).
drop-action: Use {} to indicate packet dropping.

RULES:
Each id must be unique.
Set priority values appropriately (higher for critical rules, lower for defaults). Set priority very high (e.g., 500) for queue related rules.
Don't include any **optional field** unless it is explicitly mentioned in the intent.
Use valid match conditions (ipv4-destination, tcp-destination-port, ipv4-source, in-port) depending on the intent type.
When translating HTTP, HTTPS, or other protocol traffic by port, always use 'tcp-destination-port' for the protocol's standard server port (e.g., 80 for HTTP, 443 for HTTPS) unless the intent explicitly says 'source port'.
Ensure valid ODL-compliant JSON syntax.
Avoid duplicate keys and empty fields.
Verify JSON structure for correctness before responding.
Always respond with valid JSON only, with no additional text, comments, or explanations.
If the intent cannot be mapped, return an empty JSON object {}."""

In [ ]:
num_examples = [0, 1, 3, 6, 9]

my_models = [
"marco-o1",
"mistral",
"mistral-nemo",
"deepseek-coder",
"starcoder", 
"codegemma",
"starcoder2",
"openchat",
"phi3",
"dolphin-mistral",
"wizardlm2",
"phi",
"yi",
"zephyr",
"command-r",
"llava-llama3",
"codestral",
"codellama:34b",
"codellama",
"llama2",
"llama3",
"llama3.1",
"llama3.2",
"qwen",
"qwen2",
"qwen2.5",
"gemma2:27b",
"huihui_ai/qwq-abliterated",
"huihui_ai/qwq-fusion",
"qwq"
]

#"tinyllama", "orca-mini" does not produce meaningful output

default_model = "llama2"

In [183]:
ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

In [184]:
ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

# Load custom dataset from CSV
custom_dataset = pd.read_csv('Intent2Flow-ODL.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)

# SYSTEM PROMPT (Manually define)
SYSTEM_PROMPT = TRANSLATION_PROMPT

In [185]:
def normalize_odl_value(value):
    if isinstance(value, str) and value.isdigit():
        return int(value)
    return value

def dict_equal_odl(d1, d2, ignore_fields=set()):
    """Recursively compare ODL dictionaries ignoring specified fields."""
    if isinstance(d1, dict) and isinstance(d2, dict):
        keys1 = set(d1.keys()) - ignore_fields
        keys2 = set(d2.keys()) - ignore_fields
        if keys1 != keys2:
            return False
        for k in keys1:
            if not dict_equal_odl(d1[k], d2[k], ignore_fields):
                return False
        return True
    elif isinstance(d1, list) and isinstance(d2, list):
        norm1 = sorted([normalize_odl_value(item) if not isinstance(item, dict) else item for item in d1], key=str)
        norm2 = sorted([normalize_odl_value(item) if not isinstance(item, dict) else item for item in d2], key=str)
        if len(norm1) != len(norm2):
            return False
        for x, y in zip(norm1, norm2):
            if not dict_equal_odl(x, y, ignore_fields):
                return False
        return True
    else:
        return normalize_odl_value(d1) == normalize_odl_value(d2)

def compare_odl_json(expected_json, actual_json):
    """
    Returns True if actual ODL output matches expected, ignoring irrelevant fields.
    """
    #ignore_fields = {"id", "flow-name"}  # Add more if needed, e.g., statistics fields
    ignore_fields = {"id", "flow-name", "priority", "hard-timeout", "idle-timeout"}
    
    # Defensive deep copy
    expected_json = copy.deepcopy(expected_json)
    actual_json = copy.deepcopy(actual_json)

    # Clean
    def clean(json_obj):
        for flow in json_obj.get("flow-node-inventory:flow", []):
            for field in ignore_fields:
                flow.pop(field, None)
        return json_obj
    expected_json = clean(expected_json)
    actual_json = clean(actual_json)

    # Now compare the (first) flow dicts
    ex = expected_json.get("flow-node-inventory:flow", [])[0] if expected_json.get("flow-node-inventory:flow") else {}
    ac = actual_json.get("flow-node-inventory:flow", [])[0] if actual_json.get("flow-node-inventory:flow") else {}

    return dict_equal_odl(ex, ac, ignore_fields)


In [ ]:
output_csv = "odl_translation_accuracy_results.csv"
results = []

for model in my_models:
    row = [model]
    for num_context in num_examples:
        correct_count = 0
        total_count = 0
        time_list = []
        # setup selector...
        example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
            [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
            ollama_emb,
            Chroma,
            input_keys=["instruction"],
            k=num_context,
            vectorstore_kwargs={"fetch_k": min(num_context, len(trainset))}
        )
        example_selector.vectorstore.reset_collection()
        for _, row_ in trainset.iterrows():
            example_selector.add_example({
                "instruction": row_["instruction"],
                "output": row_["output"]
            })

        for _, testcase in testset.iterrows():
            intent = testcase["instruction"]
            expected_output = testcase["output"]

            system_prompt = SYSTEM_PROMPT
            try_count = 0
            while True:
                try:
                    time.sleep(0.1)
                    t0 = time.time()
                    if num_context > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    response = client.generate(
                        model=model,
                        options={
                            'temperature': 0.6,
                            'num_ctx': 8192,
                            'top_p': 0.3,
                            'num_predict': 1024,
                            'num_gpu': 99,
                        },
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    proc_time_s = (time.time() - t0)
                    time_list.append(proc_time_s)
                    actual_output = response['response']
                    break
                except Exception as e:
                    print("Exception on Input: ", e)
                    try_count += 1
                    if try_count > 10:
                        actual_output = '{}'
                        break
                    continue

            try:
                expected_json = json.loads(expected_output)
                actual_json = json.loads(actual_output)
                total_count += 1

                result = compare_odl_json(expected_json, actual_json)

                if (not result):
                    print("\n\n\n")
                    print(expected_json)
                    print("\n")
                    print(actual_json)
                    print("\n")
                    print("Result: ", result)

                else:
                    correct_count += 1

            except Exception as e:
                print("\n\n")
                print(expected_json)
                print("\n")
                print(actual_json)
                print("\n\n")
                print("Exception found: ", e)
                total_count += 1  # still count as attempted

        accuracy = correct_count / total_count if total_count > 0 else 0
        avg_time = sum(time_list) / len(time_list) if time_list else 0
        row += [num_context, round(accuracy, 4), round(avg_time, 3)]
        print(f"Model: {model}, Context: {num_context}, Accuracy: {accuracy:.4f}, Avg Time: {avg_time:.2f}")

    results.append(row)

# Save to CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write header
    header = ["Model"]
    for num_context in num_examples:
        header += [f"context_{num_context}", f"accuracy_{num_context}", f"avg_time_{num_context}"]
    writer.writerow(header)
    for row in results:
        writer.writerow(row)

print("Results written to", output_csv)